In [1]:
import boto3
import numpy as np

In [3]:
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Titan Text Premier.
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Start a conversation with the user message.
user_message = """Your task is to analyze the following report:
<report>
{{FULL REPORT TEXT FROM https://investors.matterport.com/node/9501/html}}
</report>

Summarize this annual report in a concise and clear manner, and identify key market trends and takeaways. Output your findings as a short memo I can send to my team. The goal of the memo is to ensure my team stays up to date on how financial institutions are faring and qualitatively forecast and identify whether there are any operating and revenue risks to be expected in the coming quarter. Make sure to include all relevant details in your summary and analysis."""
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"temperature":0},
        additionalModelRequestFields={"top_k":250}
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)


Here is a concise memo summarizing the key points from Matterport's 2022 Annual Report and identifying relevant market trends and takeaways:

To: Team
From: [Your Name]
Date: [Current Date]
Subject: Summary of Matterport 2022 Annual Report

This memo provides an overview of Matterport's 2022 annual report and analysis of key market trends and risks relevant to our business.

Report Summary:
- Matterport is a spatial data company that provides 3D capture and data management solutions for the built world. Their solutions allow users to design, build, operate, promote and understand spaces in a more efficient and impactful way.
- In 2022, Matterport's total revenue was $134.7 million, up 27% year-over-year. However, they incurred a net loss of $186.2 million due to increased operating expenses.
- Subscription revenue grew 41% to $59.4 million, driven by expansion of their subscriber base to 779,000 subscribers.
- License revenue declined 5% to $48.1 million due to supply chain constraints

In [15]:
session = boto3.Session()

In [17]:
crd = session.get_credentials()